In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278


### Update QR scaling

In [195]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100
X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)
L = LinearTransform(X)
transform!(L, X);

S = Storage(C.I.f, X);

F = QRscaling(S);

newidx = [1 1 1];

In [196]:
Snew = update_storage(S, X, newidx);

In [197]:
@time Fupdated = updateQRscaling(F, Snew)
@time Fnew = QRscaling(Snew);

  0.000069 seconds (122 allocations: 14.422 KiB)
  0.000107 seconds (14 allocations: 9.406 KiB)


In [198]:
?schur

search: schur schur! Schur ordschur ordschur! GeneralizedSchur schedule



```
schur(A::StridedMatrix) -> F::Schur
```

Computes the Schur factorization of the matrix `A`. The (quasi) triangular Schur factor can be obtained from the `Schur` object `F` with either `F.Schur` or `F.T` and the orthogonal/unitary Schur vectors can be obtained with `F.vectors` or `F.Z` such that `A = F.vectors * F.Schur * F.vectors'`. The eigenvalues of `A` can be obtained with `F.values`.

Iterating the decomposition produces the components `F.T`, `F.Z`, and `F.values`.

# Examples

```jldoctest
julia> A = [5. 7.; -2. -4.]
2×2 Array{Float64,2}:
  5.0   7.0
 -2.0  -4.0

julia> F = schur(A)
Schur{Float64,Array{Float64,2}}
T factor:
2×2 Array{Float64,2}:
 3.0   9.0
 0.0  -2.0
Z factor:
2×2 Array{Float64,2}:
  0.961524  0.274721
 -0.274721  0.961524
eigenvalues:
2-element Array{Float64,1}:
  3.0
 -2.0

julia> F.vectors * F.Schur * F.vectors'
2×2 Array{Float64,2}:
  5.0   7.0
 -2.0  -4.0

julia> t, z, vals = F; # destructuring via iteration

julia> t == F.T && z == F.Z && vals == F.values
true
```

---

```
schur(A::StridedMatrix, B::StridedMatrix) -> F::GeneralizedSchur
```

Computes the Generalized Schur (or QZ) factorization of the matrices `A` and `B`. The (quasi) triangular Schur factors can be obtained from the `Schur` object `F` with `F.S` and `F.T`, the left unitary/orthogonal Schur vectors can be obtained with `F.left` or `F.Q` and the right unitary/orthogonal Schur vectors can be obtained with `F.right` or `F.Z` such that `A=F.left*F.S*F.right'` and `B=F.left*F.T*F.right'`. The generalized eigenvalues of `A` and `B` can be obtained with `F.α./F.β`.

Iterating the decomposition produces the components `F.S`, `F.T`, `F.Q`, `F.Z`, `F.α`, and `F.β`.


In [187]:
@testset "Test updateQRscaling" begin

    Nx = 3
    Ne = 8
    m = 20

    idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


    Nψ = 6
    coeff = [ 0.20649582065364197;
             -0.5150990160472986;
              2.630096893080717;
              1.13653076177397;
              0.6725837371023421;
             -1.3126095306624133]
    C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

    Ne = 300


    X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)
    L = LinearTransform(X)
    transform!(L, X)
    S = Storage(C.I.f, X)
    F = QRscaling(S)
    newidx = [1 1 1]

    Snew = update_storage(S, X, newidx)
    Fupdated = updateQRscaling(F, Snew)
    Fnew = QRscaling(Snew)
    

    # The QR decomposition is not unique! We can only check A^T*A for A ∈ {R, R^{-1}, U, U^{-1}}

    @test norm(Fupdated.D - Fnew.D)<1e-8
    @test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

    @test norm(Fupdated.R'*Fupdated.R - Fnew.R'*Fnew.R)<1e-8
    @test norm(Fupdated.Rinv'*Fupdated.Rinv - Fnew.Rinv'*Fnew.Rinv)<1e-8

    @test norm(Fupdated.Rinv.data'*Fupdated.Rinv.data - Fnew.Rinv.data'*Fnew.Rinv.data)<1e-8

    @test norm(Fupdated.U'*Fupdated.U - Fnew.U'*Fnew.U)<1e-8
    @test norm(Fupdated.Uinv'*Fupdated.Uinv - Fnew.Uinv'*Fnew.Uinv)<1e-8

    @test norm(Fupdated.L2Uinv - Fnew.L2Uinv)<1e-8
end

Test Summary:        | Pass  Total
Test updateQRscaling |    8      8


Test.DefaultTestSet("Test updateQRscaling", Any[], 8, false)

### Integration

In [24]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

LoadError: AssertionError: Size of the array of multi-indices idx is wrong

In [22]:
M = HermiteMap(m, X);

In [9]:
@time optimize(M, X, 10; P = serial);

 11.061632 seconds (49.90 M allocations: 19.060 GiB, 7.20% gc time)


In [10]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = serial, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)
